# Credit Risk Analysis: Data Preprocessing and Preparation

This notebook focuses on preparing loan data for Credit Risk Analysis. Effective data preprocessing is a cornerstone of any analytical project, and this notebook demonstrates my ability to clean, process, and transform data into a format ready for advanced analysis.

The dataset includes textual features (e.g., loan statuses, terms, and verification statuses) and numerical attributes (e.g., loan amounts, interest rates, and payment totals). Here's an overview of the preprocessing steps undertaken:

## Text Column Processing:

1. Text columns have been transformed into numeric categories wherever applicable, improving interpretability and computational efficiency.
2. Time-based columns (e.g., 'issue_d') have been mapped to numeric representations (e.g., months).

## Handling Missing Values:

1. For numeric data, the CRM rule has been applied, assuming the worst-case scenario for missing values.
2. For text data, missing values have been handled with a unique filler, ensuring no loss of interpretability.

## Currency Conversion:

1. All monetary values in USD have been converted to EUR, ensuring consistency in analysis and alignment with international perspectives.

## Column Renaming and Redundancy:

1. Columns were renamed for better clarity and usability.
2. Redundant or irrelevant columns were dropped to streamline the dataset.

## Backup Checkpoints:

1. A custom checkpoint function was developed to save intermediate stages of the processed dataset, providing a reliable fallback and efficient workflow tracking.

### This project highlights my data wrangling skills, including cleaning, transforming, and managing datasets efficiently. Additionally, the use of checkpoints demonstrates a focus on reproducibility and collaboration—key skills for any Data Analyst.

## Importing the Packages

In [1]:
# for preprocessing the data
import numpy as np
#for detecting the encoding on data
from charset_normalizer import from_path
# hide warnings
import warnings

In [2]:
# Setting up some print options
np.set_printoptions(suppress = True, linewidth = 100, precision = 2)
warnings.filterwarnings('ignore')

## Importing the Data

In [3]:
# Detect encoding
result = from_path("/kaggle/input/loan-data/loan-data.csv").best()
print(result.encoding)

cp1252


In [4]:
raw_data_np = np.genfromtxt('/kaggle/input/loan-data/loan-data.csv', 
                            delimiter = ';', 
                            encoding='cp1252', 
                            skip_header = 1, 
                            autostrip = True)
raw_data_np

array([[48010226.  ,         nan,    35000.  , ...,         nan,         nan,     9452.96],
       [57693261.  ,         nan,    30000.  , ...,         nan,         nan,     4679.7 ],
       [59432726.  ,         nan,    15000.  , ...,         nan,         nan,     1969.83],
       ...,
       [50415990.  ,         nan,    10000.  , ...,         nan,         nan,     2185.64],
       [46154151.  ,         nan,         nan, ...,         nan,         nan,     3199.4 ],
       [66055249.  ,         nan,    10000.  , ...,         nan,         nan,      301.9 ]])

## Checking for Incomplete Data

In [5]:
# get count of missing values
np.isnan(raw_data_np).sum()

88005

In [6]:
temp_fill = np.nanmax(raw_data_np) + 1
temp_mean = np.nanmean(raw_data_np, axis = 0)

# display temp_mean to get an idea on how many columns have all values missing or only text values
temp_mean

array([54015809.19,         nan,    15273.46,         nan,    15311.04,         nan,       16.62,
            440.92,         nan,         nan,         nan,         nan,         nan,     3143.85])

In [7]:
temp_stats = np.array([np.nanmin(raw_data_np, axis = 0), temp_mean, np.nanmax(raw_data_np, axis = 0)])

# getting min, mean and max for all columns
temp_stats

array([[  373332.  ,         nan,     1000.  ,         nan,     1000.  ,         nan,        6.  ,
              31.42,         nan,         nan,         nan,         nan,         nan,        0.  ],
       [54015809.19,         nan,    15273.46,         nan,    15311.04,         nan,       16.62,
             440.92,         nan,         nan,         nan,         nan,         nan,     3143.85],
       [68616519.  ,         nan,    35000.  ,         nan,    35000.  ,         nan,       28.99,
            1372.97,         nan,         nan,         nan,         nan,         nan,    41913.62]])

## Splitting the Dataset

### Splitting the Columns

In [8]:
# splitting the dataset based on datatype for ease of operation
#using temp_mean to determin text columns and columns with atleast some numerical data
columns_strings = np.argwhere(np.isnan(temp_mean)).squeeze()
columns_strings

array([ 1,  3,  5,  8,  9, 10, 11, 12])

In [9]:
columns_numeric = np.argwhere(np.isnan(temp_mean) == False).squeeze()
columns_numeric

array([ 0,  2,  4,  6,  7, 13])

### Re-importing the Dataset

In [10]:
# re-importing the text columns separately
loan_data_strings = np.genfromtxt('/kaggle/input/loan-data/loan-data.csv', 
                            delimiter = ';', 
                            encoding='cp1252',
                            usecols = columns_strings,
                            skip_header = 1,
                            dtype = str,
                            autostrip = True)
loan_data_strings

array([['May-15', 'Current', '36 months', ..., 'Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=48010226', 'CA'],
       ['', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=57693261', 'NY'],
       ['Sep-15', 'Current', '36 months', ..., 'Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=59432726', 'PA'],
       ...,
       ['Jun-15', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=50415990', 'CA'],
       ['Apr-15', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=46154151', 'OH'],
       ['Dec-15', 'Current', '36 months', ..., '',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=66055249', 'IL']],
      dtype='<U69')

In [11]:
# re-importing the numeric columns separately
# re-importing the text columns separately
loan_data_numeric = np.genfromtxt('/kaggle/input/loan-data/loan-data.csv', 
                            delimiter = ';', 
                            encoding='cp1252',
                            usecols = columns_numeric,
                            filling_values = temp_fill,
                            skip_header = 1,
                            autostrip = True)
loan_data_numeric

array([[48010226.  ,    35000.  ,    35000.  ,       13.33,     1184.86,     9452.96],
       [57693261.  ,    30000.  ,    30000.  , 68616520.  ,      938.57,     4679.7 ],
       [59432726.  ,    15000.  ,    15000.  , 68616520.  ,      494.86,     1969.83],
       ...,
       [50415990.  ,    10000.  ,    10000.  , 68616520.  , 68616520.  ,     2185.64],
       [46154151.  , 68616520.  ,    10000.  ,       16.55,      354.3 ,     3199.4 ],
       [66055249.  ,    10000.  ,    10000.  , 68616520.  ,      309.97,      301.9 ]])

### The Names of the Columns

In [12]:
header_full = np.genfromtxt('/kaggle/input/loan-data/loan-data.csv', 
                            delimiter = ';', 
                            encoding='cp1252',
                            skip_footer = raw_data_np.shape[0],
                            dtype = str,
                            autostrip = True)
header_full

array(['id', 'issue_d', 'loan_amnt', 'loan_status', 'funded_amnt', 'term', 'int_rate',
       'installment', 'grade', 'sub_grade', 'verification_status', 'url', 'addr_state',
       'total_pymnt'], dtype='<U19')

In [13]:
header_strings, header_numeric = header_full[columns_strings], header_full[columns_numeric]

In [14]:
header_strings

array(['issue_d', 'loan_status', 'term', 'grade', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

In [15]:
header_numeric

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt'], dtype='<U19')

## Creating Checkpoints:

In [16]:
# creating checkpoint files for recovery
def checkpoint(file_name, checkpoint_header, checkpoint_data):
    np.savez(file_name, header = checkpoint_header, data = checkpoint_data)
    checkpoint_variable = np.load(file_name + ".npz")
    return(checkpoint_variable)

In [17]:
checkpoint_1_strings = checkpoint('checkpoint-1-strings', header_strings, loan_data_strings)
checkpoint_1_strings['header']

array(['issue_d', 'loan_status', 'term', 'grade', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

In [18]:
checkpoint_1_numeric = checkpoint('checkpoint-1-numeric', header_numeric, loan_data_numeric)
checkpoint_1_numeric['header']

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt'], dtype='<U19')

## Manipulating String Columns

In [19]:
loan_data_strings = checkpoint_1_strings['data']
header_strings = checkpoint_1_strings['header']

In [20]:
# updating column names to be more meaningful
print(header_strings)
header_strings[0] = 'issue_date'
print(header_strings)

['issue_d' 'loan_status' 'term' 'grade' 'sub_grade' 'verification_status' 'url' 'addr_state']
['issue_date' 'loan_status' 'term' 'grade' 'sub_grade' 'verification_status' 'url' 'addr_state']


### Issue Date

In [21]:
# since all loan data is from 2015, we can remove it without losing information
print(np.unique(loan_data_strings[:, 0]))
loan_data_strings[:, 0] = np.chararray.strip(loan_data_strings[:, 0], '-15')
print(np.unique(loan_data_strings[:, 0]))

['' 'Apr-15' 'Aug-15' 'Dec-15' 'Feb-15' 'Jan-15' 'Jul-15' 'Jun-15' 'Mar-15' 'May-15' 'Nov-15'
 'Oct-15' 'Sep-15']
['' 'Apr' 'Aug' 'Dec' 'Feb' 'Jan' 'Jul' 'Jun' 'Mar' 'May' 'Nov' 'Oct' 'Sep']


In [22]:
# for analysis we can substitute month name with numbers, this helps reduce data size
months = np.array(['', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
for i in range(13):
        loan_data_strings[:,0] = np.where(loan_data_strings[:,0] == months[i],
                                          i,
                                          loan_data_strings[:,0])

In [23]:
print(np.unique(loan_data_strings[:, 0]))

['0' '1' '10' '11' '12' '2' '3' '4' '5' '6' '7' '8' '9']


### Loan Status

In [24]:
print(header_strings)

['issue_date' 'loan_status' 'term' 'grade' 'sub_grade' 'verification_status' 'url' 'addr_state']


In [25]:
print(np.unique(loan_data_strings[:, 1]))

['' 'Charged Off' 'Current' 'Default' 'Fully Paid' 'In Grace Period' 'Issued' 'Late (16-30 days)'
 'Late (31-120 days)']


In [26]:
# for the usecase we only care if the candidate is in a stable financial condition, hence loan status can be a simple flag value
# (1) good = current, issed, fully paid, in grace period, late (16-30 days)(short latency)
# (0) bad = '', charged off, default, late (16-30 days)(short latency)

good_statuses = np.array(['Current', 'Fully Paid', 'In Grace Period', 'Issued', 'Late (16-30 days)'])

loan_data_strings[:, 1] = np.where(np.isin(loan_data_strings[:, 1], good_statuses),
                                   1,
                                   0)

print(np.unique(loan_data_strings[:, 1]))

['0' '1']


### Term

In [27]:
print(header_strings)

['issue_date' 'loan_status' 'term' 'grade' 'sub_grade' 'verification_status' 'url' 'addr_state']


In [28]:
print(np.unique(loan_data_strings[:, 2]))

# based on above output months is redundant in this data so we can strip it 
loan_data_strings[:, 2] = np.chararray.strip(loan_data_strings[:, 2], ' months')
print(np.unique(loan_data_strings[:, 2]))

# make the column name more fitting to our new data
header_strings[2] = 'term-months'

['' '36 months' '60 months']
['' '36' '60']


In [29]:
# in credit risk model we have to assume worst for missing data, hence replacing all empty data for column - term months with 60
loan_data_strings[:, 2] = np.where(loan_data_strings[:, 2] == '', '60', loan_data_strings[:, 2])
print(np.unique(loan_data_strings[:, 2]))

['36' '60']


### Grade and Subgrade

In [30]:
print(header_strings)

['issue_date' 'loan_status' 'term-months' 'grade' 'sub_grade' 'verification_status' 'url'
 'addr_state']


In [31]:
print(np.unique(loan_data_strings[:, 3]))
print(np.unique(loan_data_strings[:, 4]))

['' 'A' 'B' 'C' 'D' 'E' 'F' 'G']
['' 'A1' 'A2' 'A3' 'A4' 'A5' 'B1' 'B2' 'B3' 'B4' 'B5' 'C1' 'C2' 'C3' 'C4' 'C5' 'D1' 'D2' 'D3' 'D4'
 'D5' 'E1' 'E2' 'E3' 'E4' 'E5' 'F1' 'F2' 'F3' 'F4' 'F5' 'G1' 'G2' 'G3' 'G4' 'G5']


#### Filling Sub Grade

In [32]:
# based on above ouptput for most cases, grade is redundant as sub-grade offer comprehensive information. 
# but in cases where sub-grade is not available as per the thumbrule of CRM we can assign the worst subgrade based on its grade

for i in np.unique(loan_data_strings[:,3])[1:]:
    loan_data_strings[:,4] = np.where((loan_data_strings[:,4] == '') & (loan_data_strings[:,3] == i),
                                      i + '5',
                                      loan_data_strings[:,4])
print(np.unique(loan_data_strings[:, 4],return_counts = True))

(array(['', 'A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4',
       'C5', 'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4',
       'F5', 'G1', 'G2', 'G3', 'G4', 'G5'], dtype='<U69'), array([  9, 285, 278, 239, 323, 592, 509, 517, 530, 553, 633, 629, 567, 586, 564, 577, 391, 267,
       250, 255, 288, 235, 162, 171, 139, 160,  94,  52,  34,  43,  24,  19,  10,   3,   7,   5]))


In [33]:
# for empty values still left in subgrade column based on CRM requirement we assign a bad grade H1 which will be unique.
# having a unique grade for such cases will help the model identify cases where data has been intentionally withheld by the candidate
loan_data_strings[:, 4] = np.where((loan_data_strings[:, 4] == ''), 'H1', loan_data_strings[:, 4])
print(np.unique(loan_data_strings[:, 4],return_counts = True))

(array(['A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4', 'C5',
       'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4', 'F5',
       'G1', 'G2', 'G3', 'G4', 'G5', 'H1'], dtype='<U69'), array([285, 278, 239, 323, 592, 509, 517, 530, 553, 633, 629, 567, 586, 564, 577, 391, 267, 250,
       255, 288, 235, 162, 171, 139, 160,  94,  52,  34,  43,  24,  19,  10,   3,   7,   5,   9]))


#### Removing Grade

In [34]:
loan_data_strings = np.delete(loan_data_strings, 3, axis = 1)

In [35]:
print(loan_data_strings[:, 3])

['C3' 'A5' 'B5' ... 'A5' 'D2' 'A4']


In [36]:
header_strings = np.delete(header_strings, 3)

In [37]:
print(header_strings)

['issue_date' 'loan_status' 'term-months' 'sub_grade' 'verification_status' 'url' 'addr_state']


#### Converting Sub Grade

In [38]:
print(np.unique(loan_data_strings[:, 3]))

['A1' 'A2' 'A3' 'A4' 'A5' 'B1' 'B2' 'B3' 'B4' 'B5' 'C1' 'C2' 'C3' 'C4' 'C5' 'D1' 'D2' 'D3' 'D4'
 'D5' 'E1' 'E2' 'E3' 'E4' 'E5' 'F1' 'F2' 'F3' 'F4' 'F5' 'G1' 'G2' 'G3' 'G4' 'G5' 'H1']


In [39]:
# to convert the subgrade into a column with only numbers we use a dictionary that has keys as the subgrade 
# and values as a list of numbers starting with 1 till count of unique subgrades + 1 (range is exclusive of upper limit)

keys = list(np.unique(loan_data_strings[:,3]))                         
values = list(range(1, np.unique(loan_data_strings[:,3]).shape[0] + 1)) 
dict_sub_grade = dict(zip(keys, values))
print(dict_sub_grade)

{'A1': 1, 'A2': 2, 'A3': 3, 'A4': 4, 'A5': 5, 'B1': 6, 'B2': 7, 'B3': 8, 'B4': 9, 'B5': 10, 'C1': 11, 'C2': 12, 'C3': 13, 'C4': 14, 'C5': 15, 'D1': 16, 'D2': 17, 'D3': 18, 'D4': 19, 'D5': 20, 'E1': 21, 'E2': 22, 'E3': 23, 'E4': 24, 'E5': 25, 'F1': 26, 'F2': 27, 'F3': 28, 'F4': 29, 'F5': 30, 'G1': 31, 'G2': 32, 'G3': 33, 'G4': 34, 'G5': 35, 'H1': 36}


In [40]:
for i in np.unique(loan_data_strings[:,3]):
        loan_data_strings[:,3] = np.where(loan_data_strings[:,3] == i, 
                                          dict_sub_grade[i],
                                          loan_data_strings[:,3])

In [41]:
print(np.unique(loan_data_strings[:, 3]))

['1' '10' '11' '12' '13' '14' '15' '16' '17' '18' '19' '2' '20' '21' '22' '23' '24' '25' '26' '27'
 '28' '29' '3' '30' '31' '32' '33' '34' '35' '36' '4' '5' '6' '7' '8' '9']


### Verification Status

In [42]:
print(header_strings)

['issue_date' 'loan_status' 'term-months' 'sub_grade' 'verification_status' 'url' 'addr_state']


In [43]:
print(np.unique(loan_data_strings[:, 4]))

['' 'Not Verified' 'Source Verified' 'Verified']


In [44]:
# as a categorical solumn we can narrow the column down to only flag 0,1 values
# (1) good = Verified, Source Verified
# (0) bad = '', Not Verified

verfication_status_good = np.array(['Verified', 'Source Verified'])

loan_data_strings[:, 4] = np.where(np.isin(loan_data_strings[:, 4], verfication_status_good),
                                   1,
                                   0)
print(np.unique(loan_data_strings[:, 4]))

['0' '1']


### URL

In [45]:
print(header_strings)

['issue_date' 'loan_status' 'term-months' 'sub_grade' 'verification_status' 'url' 'addr_state']


In [46]:
print(np.unique(loan_data_strings[:, 5]))

['https://www.lendingclub.com/browse/loanDetail.action?loan_id=12606806'
 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=13026045'
 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=1312426' ...
 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=8138291'
 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=8214572'
 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=849994']


In [47]:
# since the first part of the url is common for all data, we can checkout the output after stripping the repetative data
loan_data_strings[:, 5] = np.chararray.strip(loan_data_strings[:, 5], 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=')

print(np.unique(loan_data_strings[:, 5]))

['12606806' '13026045' '1312426' ... '8138291' '8214572' '849994']


In [48]:
# post stripping the output url column now looks like the numeric id column. lets confirm the same before deleting the url column
print(np.unique(loan_data_numeric[:,0].astype(dtype = np.int32) == loan_data_strings[:, 5].astype(dtype = np.int32)))

[ True]


In [49]:
# deleting the url column
loan_data_strings = np.delete(loan_data_strings, 5, axis = 1)
header_strings = np.delete(header_strings, 5)

In [50]:
print(loan_data_strings[:, 5])
print(header_strings)

['CA' 'NY' 'PA' ... 'CA' 'OH' 'IL']
['issue_date' 'loan_status' 'term-months' 'sub_grade' 'verification_status' 'addr_state']


In [51]:
print(loan_data_numeric[:, 0])
print(header_numeric)

[48010226. 57693261. 59432726. ... 50415990. 46154151. 66055249.]
['id' 'loan_amnt' 'funded_amnt' 'int_rate' 'installment' 'total_pymnt']


### State Address

In [52]:
print(header_strings)

['issue_date' 'loan_status' 'term-months' 'sub_grade' 'verification_status' 'addr_state']


In [53]:
print(np.unique(loan_data_strings[:, 5], return_counts = True))
print(np.unique(loan_data_strings[:, 5]).size)

# USA has 50 states and above output does give 50 values but one is a missing scenario. 
# From the output we can determine that the only state not available in data in Iowa (IA), so we can use this as benchmark

(array(['', 'AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'IL', 'IN',
       'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH',
       'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA',
       'VT', 'WA', 'WI', 'WV', 'WY'], dtype='<U69'), array([ 500,   26,  119,   74,  220, 1336,  201,  143,   27,   27,  690,  321,   44,  389,  152,
         84,   84,  116,  210,  222,   10,  267,  156,  160,   61,   28,  261,   16,   25,   58,
        341,   57,  130,  777,  312,   83,  108,  320,   40,  107,   24,  143,  758,   74,  242,
         17,  216,  148,   49,   27]))
50


In [54]:
states_names, states_count = np.unique(loan_data_strings[:, 5], return_counts = True)
states_count_sorted = np.argsort(-states_count) # - indicates we are sorting the data in descending order
print(states_names[states_count_sorted], states_count[states_count_sorted])

['CA' 'NY' 'TX' 'FL' '' 'IL' 'NJ' 'GA' 'PA' 'OH' 'MI' 'NC' 'VA' 'MD' 'AZ' 'WA' 'MA' 'CO' 'MO' 'MN'
 'IN' 'WI' 'CT' 'TN' 'NV' 'AL' 'LA' 'OR' 'SC' 'KY' 'KS' 'OK' 'UT' 'AR' 'MS' 'NH' 'NM' 'WV' 'HI'
 'RI' 'MT' 'DE' 'DC' 'WY' 'AK' 'NE' 'SD' 'VT' 'ND' 'ME'] [1336  777  758  690  500  389  341  321  320  312  267  261  242  222  220  216  210  201  160
  156  152  148  143  143  130  119  116  108  107   84   84   83   74   74   61   58   57   49
   44   40   28   27   27   27   26   25   24   17   16   10]


In [55]:
# Handling the missing values
loan_data_strings[:, 5] = np.where(loan_data_strings[:, 5] == '', 
                                   0, 
                                   loan_data_strings[:, 5])

print(np.unique(loan_data_strings[:, 5]))

['0' 'AK' 'AL' 'AR' 'AZ' 'CA' 'CO' 'CT' 'DC' 'DE' 'FL' 'GA' 'HI' 'IL' 'IN' 'KS' 'KY' 'LA' 'MA' 'MD'
 'ME' 'MI' 'MN' 'MO' 'MS' 'MT' 'NC' 'ND' 'NE' 'NH' 'NJ' 'NM' 'NV' 'NY' 'OH' 'OK' 'OR' 'PA' 'RI'
 'SC' 'SD' 'TN' 'TX' 'UT' 'VA' 'VT' 'WA' 'WI' 'WV' 'WY']


In [56]:
# Since we have little data for too many states, assign unique value to each state will allow outliers to have a big effect on coefficients.
# Hence we group the states as per geographical region

states_west = np.array(['WA', 'OR','CA','NV','ID','MT', 'WY','UT','CO', 'AZ','NM','HI','AK'])
states_south = np.array(['TX','OK','AR','LA','MS','AL','TN','KY','FL','GA','SC','NC','VA','WV','MD','DE','DC'])
states_midwest = np.array(['ND','SD','NE','KS','MN','IA','MO','WI','IL','IN','MI','OH'])
states_east = np.array(['PA','NY','NJ','CT','MA','VT','NH','ME','RI'])

https://www2.census.gov/geo/pdfs/maps-data/maps/reference/us_regdiv.pdf

In [57]:
loan_data_strings[:, 5] = np.where(np.isin(loan_data_strings[:, 5], states_west),1,loan_data_strings[:, 5])
loan_data_strings[:, 5] = np.where(np.isin(loan_data_strings[:, 5], states_south),2,loan_data_strings[:, 5])
loan_data_strings[:, 5] = np.where(np.isin(loan_data_strings[:, 5], states_midwest),3,loan_data_strings[:, 5])
loan_data_strings[:, 5] = np.where(np.isin(loan_data_strings[:, 5], states_east),4,loan_data_strings[:, 5])
print(np.unique(loan_data_strings[:, 5]))

['0' '1' '2' '3' '4']


## Converting to Numbers

In [58]:
# all strings in this dataset has been updated to look like numbers, so we can now update the dtype to int

loan_data_strings

array([['5', '1', '36', '13', '1', '1'],
       ['0', '1', '36', '5', '1', '4'],
       ['9', '1', '36', '10', '1', '4'],
       ...,
       ['6', '1', '36', '5', '1', '1'],
       ['4', '1', '36', '17', '1', '3'],
       ['12', '1', '36', '4', '0', '3']], dtype='<U69')

In [59]:
loan_data_strings = loan_data_strings.astype(int)

In [60]:
print(loan_data_strings)

[[ 5  1 36 13  1  1]
 [ 0  1 36  5  1  4]
 [ 9  1 36 10  1  4]
 ...
 [ 6  1 36  5  1  1]
 [ 4  1 36 17  1  3]
 [12  1 36  4  0  3]]


### Checkpoint 2: Strings

In [61]:
checkpoint_2_strings = checkpoint('checkpoint-2-strings', header_strings, loan_data_strings)

In [62]:
print(checkpoint_2_strings['header'])
print(checkpoint_2_strings['data'])

['issue_date' 'loan_status' 'term-months' 'sub_grade' 'verification_status' 'addr_state']
[[ 5  1 36 13  1  1]
 [ 0  1 36  5  1  4]
 [ 9  1 36 10  1  4]
 ...
 [ 6  1 36  5  1  1]
 [ 4  1 36 17  1  3]
 [12  1 36  4  0  3]]


In [63]:
np.array_equal(loan_data_strings, checkpoint_2_strings['data'])

True

## Manipulating Numeric Columns

In [64]:
loan_data_numeric

array([[48010226.  ,    35000.  ,    35000.  ,       13.33,     1184.86,     9452.96],
       [57693261.  ,    30000.  ,    30000.  , 68616520.  ,      938.57,     4679.7 ],
       [59432726.  ,    15000.  ,    15000.  , 68616520.  ,      494.86,     1969.83],
       ...,
       [50415990.  ,    10000.  ,    10000.  , 68616520.  , 68616520.  ,     2185.64],
       [46154151.  , 68616520.  ,    10000.  ,       16.55,      354.3 ,     3199.4 ],
       [66055249.  ,    10000.  ,    10000.  , 68616520.  ,      309.97,      301.9 ]])

In [65]:
# this is 0 as while importing we have used a temporary fill
np.isnan(loan_data_numeric).sum()

0

### Substitute "Filler" Values

In [66]:
# Below we will individually handle missing values in each column and replace it with the worst possible outcome as per the CRM thumbrule
header_numeric

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt'], dtype='<U19')

#### ID

In [67]:
temp_fill

68616520.0

In [68]:
np.isin(loan_data_numeric[:, 0], temp_fill).sum()
# that means first column does not have any filler values

0

#### Temporary Stats

In [69]:
# for each column in header_numric the worst outcome will either be min or max of the column -
# min - 'funded_amnt'
# max - 'id', 'loan_amnt', 'int_rate', 'installment', 'total_pymnt'
temp_stats[:, columns_numeric]

array([[  373332.  ,     1000.  ,     1000.  ,        6.  ,       31.42,        0.  ],
       [54015809.19,    15273.46,    15311.04,       16.62,      440.92,     3143.85],
       [68616519.  ,    35000.  ,    35000.  ,       28.99,     1372.97,    41913.62]])

#### Funded Amount

In [70]:
temp_stats[0, columns_numeric[2]]

1000.0

In [71]:
# setting missing values in funded_amt column to min of the column
loan_data_numeric[:,2] = np.where(loan_data_numeric[:,2] == temp_fill, 
                                  temp_stats[0, columns_numeric[2]],
                                  loan_data_numeric[:,2])
loan_data_numeric[:,2]

array([35000., 30000., 15000., ..., 10000., 10000., 10000.])

In [72]:
np.isin(loan_data_numeric[:, 2], temp_fill).sum()

0

#### Loaned Amount, Interest Rate, Total Payment, Installment

In [73]:
temp_stats[2, columns_numeric]
header_numeric

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt'], dtype='<U19')

In [74]:
print(np.isin(loan_data_numeric[:, 1], temp_fill).sum())
print(np.isin(loan_data_numeric[:, 3], temp_fill).sum())
print(np.isin(loan_data_numeric[:, 4], temp_fill).sum())
print(np.isin(loan_data_numeric[:, 5], temp_fill).sum())

500
6004
501
500


In [75]:
for i in [1,3,4,5]:
    loan_data_numeric[:, i] = np.where(np.isin(loan_data_numeric[:, i], temp_fill), 
                                        temp_stats[2, columns_numeric[i]],
                                        loan_data_numeric[:, i])

In [76]:
print(np.isin(loan_data_numeric[:, 1], temp_fill).sum())
print(np.isin(loan_data_numeric[:, 3], temp_fill).sum())
print(np.isin(loan_data_numeric[:, 4], temp_fill).sum())
print(np.isin(loan_data_numeric[:, 5], temp_fill).sum())

0
0
0
0


### Currency Change

#### The Exchange Rate

In [77]:
# Our file contains exchange rates of each month of 2015, which can be used for USD to EUR conversion since all pur loan data is from 2015
EUR_USD = np.genfromtxt('/kaggle/input/monthly-exchange-rate-eur-to-usd/EUR-USD.csv', 
                        delimiter = ',', 
                        autostrip = True, 
                        skip_header = True,
                       usecols = 3) # from the dataset we only need the adjusted closing rate with is in the 4th column
EUR_USD

array([1.13, 1.12, 1.08, 1.11, 1.1 , 1.12, 1.09, 1.13, 1.13, 1.1 , 1.06, 1.09])

In [78]:
# This is our issue_date column which was originally numeric data but now has been converted to the month number instead of year
loan_data_strings[:, 0]
exchange_rate = loan_data_strings[:, 0]
exchange_rate

array([ 5,  0,  9, ...,  6,  4, 12])

In [79]:
# for all months
for i in range(1,13):
    exchange_rate = np.where(exchange_rate == i,
                             EUR_USD[i-1],
                             exchange_rate)    

# for missing data which has been substituted with 0, we use the mean of all the monthly exchange rates
exchange_rate = np.where(exchange_rate == 0,
                             np.mean(EUR_USD),
                             exchange_rate)  

exchange_rate

array([1.1 , 1.11, 1.13, ..., 1.12, 1.11, 1.09])

In [80]:
# check shapes for compatibility
print(exchange_rate.shape)
print(loan_data_strings.shape)

(10000,)
(10000, 6)


In [81]:
# using reshape to make them compatible
exchange_rate = np.reshape(exchange_rate, (10000, 1))
print(exchange_rate, exchange_rate.shape)

[[1.1 ]
 [1.11]
 [1.13]
 ...
 [1.12]
 [1.11]
 [1.09]] (10000, 1)


In [82]:
# using stacking to add the column to our dataset
loan_data_numeric = np.hstack((loan_data_numeric, exchange_rate))
print(loan_data_numeric)

[[48010226.      35000.      35000.   ...     1184.86     9452.96        1.1 ]
 [57693261.      30000.      30000.   ...      938.57     4679.7         1.11]
 [59432726.      15000.      15000.   ...      494.86     1969.83        1.13]
 ...
 [50415990.      10000.      10000.   ...     1372.97     2185.64        1.12]
 [46154151.      35000.      10000.   ...      354.3      3199.4         1.11]
 [66055249.      10000.      10000.   ...      309.97      301.9         1.09]]


In [83]:
# adding the new column name to header as well
header_numeric = np.concatenate((header_numeric, np.array(['exchange_rate'])))

In [84]:
header_numeric

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt', 'exchange_rate'],
      dtype='<U19')

#### From USD to EUR

In [85]:
# for conversion
columns_dollar = np.array([1,2,4,5])
loan_data_numeric[:, columns_dollar]

array([[35000.  , 35000.  ,  1184.86,  9452.96],
       [30000.  , 30000.  ,   938.57,  4679.7 ],
       [15000.  , 15000.  ,   494.86,  1969.83],
       ...,
       [10000.  , 10000.  ,  1372.97,  2185.64],
       [35000.  , 10000.  ,   354.3 ,  3199.4 ],
       [10000.  , 10000.  ,   309.97,   301.9 ]])

In [86]:
# get the stored exchange rate 
loan_data_numeric[:, 6]

array([1.1 , 1.11, 1.13, ..., 1.12, 1.11, 1.09])

In [87]:
# performing the calculation of USD multiplied by exchange rate for each column and stacking the result columns to our dataset
for i in columns_dollar:
    loan_data_numeric = np.hstack((loan_data_numeric, np.reshape(loan_data_numeric[:,i] / loan_data_numeric[:,6], (10000,1))))

loan_data_numeric

array([[48010226.  ,    35000.  ,    35000.  , ...,    31933.3 ,     1081.04,     8624.69],
       [57693261.  ,    30000.  ,    30000.  , ...,    27132.46,      848.86,     4232.39],
       [59432726.  ,    15000.  ,    15000.  , ...,    13326.3 ,      439.64,     1750.04],
       ...,
       [50415990.  ,    10000.  ,    10000.  , ...,     8910.3 ,     1223.36,     1947.47],
       [46154151.  ,    35000.  ,    10000.  , ...,     8997.4 ,      318.78,     2878.63],
       [66055249.  ,    10000.  ,    10000.  , ...,     9145.8 ,      283.49,      276.11]])

#### Expanding the header

In [88]:
# creating column names for the new columns housing EUR amounts
header_additional = np.array([column_name + '_EUR' for column_name in header_numeric[columns_dollar]])
header_additional

array(['loan_amnt_EUR', 'funded_amnt_EUR', 'installment_EUR', 'total_pymnt_EUR'], dtype='<U15')

In [89]:
header_numeric = np.concatenate((header_numeric, header_additional))

In [90]:
header_numeric

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt', 'exchange_rate',
       'loan_amnt_EUR', 'funded_amnt_EUR', 'installment_EUR', 'total_pymnt_EUR'], dtype='<U19')

In [91]:
# updating names for existing columns to indicate explicitly their currency
header_numeric[columns_dollar] = np.array([column_name + '_USD' for column_name in header_numeric[columns_dollar]])

In [92]:
header_numeric

array(['id', 'loan_amnt_USD', 'funded_amnt_USD', 'int_rate', 'installment_USD', 'total_pymnt_USD',
       'exchange_rate', 'loan_amnt_EUR', 'funded_amnt_EUR', 'installment_EUR', 'total_pymnt_EUR'],
      dtype='<U19')

In [93]:
# creating a new column order so that each usd column is followed by its eur counterpart
columns_index_order = [0,1,7,2,8,3,4,9,5,10,6]

In [94]:
header_numeric = header_numeric[columns_index_order]

In [95]:
header_numeric

array(['id', 'loan_amnt_USD', 'loan_amnt_EUR', 'funded_amnt_USD', 'funded_amnt_EUR', 'int_rate',
       'installment_USD', 'installment_EUR', 'total_pymnt_USD', 'total_pymnt_EUR', 'exchange_rate'],
      dtype='<U19')

In [96]:
loan_data_numeric = loan_data_numeric[:, columns_index_order]

In [97]:
loan_data_numeric

array([[48010226.  ,    35000.  ,    31933.3 , ...,     9452.96,     8624.69,        1.1 ],
       [57693261.  ,    30000.  ,    27132.46, ...,     4679.7 ,     4232.39,        1.11],
       [59432726.  ,    15000.  ,    13326.3 , ...,     1969.83,     1750.04,        1.13],
       ...,
       [50415990.  ,    10000.  ,     8910.3 , ...,     2185.64,     1947.47,        1.12],
       [46154151.  ,    35000.  ,    31490.9 , ...,     3199.4 ,     2878.63,        1.11],
       [66055249.  ,    10000.  ,     9145.8 , ...,      301.9 ,      276.11,        1.09]])

### Interest Rate

In [98]:
header_numeric

array(['id', 'loan_amnt_USD', 'loan_amnt_EUR', 'funded_amnt_USD', 'funded_amnt_EUR', 'int_rate',
       'installment_USD', 'installment_EUR', 'total_pymnt_USD', 'total_pymnt_EUR', 'exchange_rate'],
      dtype='<U19')

In [99]:
loan_data_numeric[:, 5]

array([13.33, 28.99, 28.99, ..., 28.99, 16.55, 28.99])

In [100]:
# converting the interest rate column from percentages to their counterpart floats between 0 - 1
loan_data_numeric[:, 5] = loan_data_numeric[:, 5] / 100

In [101]:
loan_data_numeric[:, 5]

array([0.13, 0.29, 0.29, ..., 0.29, 0.17, 0.29])

### Checkpoint 3: Numeric

In [102]:
checkpoint_2_numeric = checkpoint('checkpoint-2-numeric', header_numeric, loan_data_numeric)

In [103]:
checkpoint_2_numeric['header'], checkpoint_2_numeric['data']

(array(['id', 'loan_amnt_USD', 'loan_amnt_EUR', 'funded_amnt_USD', 'funded_amnt_EUR', 'int_rate',
        'installment_USD', 'installment_EUR', 'total_pymnt_USD', 'total_pymnt_EUR', 'exchange_rate'],
       dtype='<U19'),
 array([[48010226.  ,    35000.  ,    31933.3 , ...,     9452.96,     8624.69,        1.1 ],
        [57693261.  ,    30000.  ,    27132.46, ...,     4679.7 ,     4232.39,        1.11],
        [59432726.  ,    15000.  ,    13326.3 , ...,     1969.83,     1750.04,        1.13],
        ...,
        [50415990.  ,    10000.  ,     8910.3 , ...,     2185.64,     1947.47,        1.12],
        [46154151.  ,    35000.  ,    31490.9 , ...,     3199.4 ,     2878.63,        1.11],
        [66055249.  ,    10000.  ,     9145.8 , ...,      301.9 ,      276.11,        1.09]]))

## Creating the "Complete" Dataset

In [106]:
print(checkpoint_2_strings['data'].shape, checkpoint_2_numeric['data'].shape)

(10000, 6) (10000, 11)


In [109]:
loan_data = np.hstack((checkpoint_2_numeric['data'], checkpoint_2_strings['data']))

In [110]:
print(loan_data, loan_data.shape)

[[48010226.      35000.      31933.3  ...       13.          1.          1.  ]
 [57693261.      30000.      27132.46 ...        5.          1.          4.  ]
 [59432726.      15000.      13326.3  ...       10.          1.          4.  ]
 ...
 [50415990.      10000.       8910.3  ...        5.          1.          1.  ]
 [46154151.      35000.      31490.9  ...       17.          1.          3.  ]
 [66055249.      10000.       9145.8  ...        4.          0.          3.  ]] (10000, 17)


In [111]:
np.isnan(loan_data).sum()

0

In [113]:
header_full = np.concatenate((checkpoint_2_numeric['header'], checkpoint_2_strings['header']))

In [114]:
print(header_full, header_full.shape)

['id' 'loan_amnt_USD' 'loan_amnt_EUR' 'funded_amnt_USD' 'funded_amnt_EUR' 'int_rate'
 'installment_USD' 'installment_EUR' 'total_pymnt_USD' 'total_pymnt_EUR' 'exchange_rate'
 'issue_date' 'loan_status' 'term-months' 'sub_grade' 'verification_status' 'addr_state'] (17,)


## Sorting the New Dataset

In [118]:
# re-arranging all the dataset with respect to id
loan_data = loan_data[np.argsort(loan_data[:, 0])]

In [119]:
loan_data

array([[  373332.  ,     9950.  ,     9038.08, ...,       21.  ,        0.  ,        1.  ],
       [  575239.  ,    12000.  ,    10900.2 , ...,       25.  ,        1.  ,        2.  ],
       [  707689.  ,    10000.  ,     8924.3 , ...,       13.  ,        1.  ,        0.  ],
       ...,
       [68614880.  ,     5600.  ,     5121.65, ...,        8.  ,        1.  ,        1.  ],
       [68615915.  ,     4000.  ,     3658.32, ...,       10.  ,        1.  ,        2.  ],
       [68616519.  ,    21600.  ,    19754.93, ...,        3.  ,        0.  ,        2.  ]])

In [121]:
np.argsort(loan_data[:, 0])

array([   0,    1,    2, ..., 9997, 9998, 9999])

## Storing the New Dataset

In [125]:
# ading the header to the dataset
loan_data = np.vstack((header_full, loan_data))

In [126]:
loan_data

array([['id', 'loan_amnt_USD', 'loan_amnt_EUR', ..., 'sub_grade', 'verification_status',
        'addr_state'],
       ['373332.0', '9950.0', '9038.082814338286', ..., '21.0', '0.0', '1.0'],
       ['575239.0', '12000.0', '10900.20037910145', ..., '25.0', '1.0', '2.0'],
       ...,
       ['68614880.0', '5600.0', '5121.647851612413', ..., '8.0', '1.0', '1.0'],
       ['68615915.0', '4000.0', '3658.319894008867', ..., '10.0', '1.0', '2.0'],
       ['68616519.0', '21600.0', '19754.927427647883', ..., '3.0', '0.0', '2.0']], dtype='<U32')

In [128]:
np.savetxt('loan_data_preprocessed.csv', loan_data, fmt = '%s', delimiter = ',')